In [48]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import trange


import flygym.common
import flygym.mujoco
import flygym.mujoco.preprogrammed

from flygym.mujoco import Parameters

from nmf_project import (
    NeuromechflyProject,
    all_actuated_dof,)


from scipy.signal import find_peaks, medfilt
from IPython.display import Video

from flygym.mujoco.examples.rule_based_controller import PreprogrammedSteps

In [49]:
# Joint angles according to indices
LF_COXA = 0
LF_COXA_ROLL = 1
LF_COXA_YAW = 2
LF_FEMUR = 3
LF_FEMUR_ROLL = 4
LF_TIBIA = 5
LF_TARSUS = 6

LM_COXA = 7
LM_COXA_ROLL = 8
LM_COXA_YAW = 9
LM_FEMUR = 10
LM_FEMUR_ROLL = 11
LM_TIBIA = 12
LM_TARSUS = 13

LH_COXA = 14
LH_COXA_ROLL = 15
LH_COXA_YAW = 16
LH_FEMUR = 17
LH_FEMUR_ROLL = 18
LH_TIBIA = 19
LH_TARSUS = 20

RF_COXA = 21
RF_COXA_ROLL = 22
RF_COXA_YAW = 23
RF_FEMUR = 24
RF_FEMUR_ROLL = 25
RF_TIBIA = 26
RF_TARSUS = 27

RM_COXA = 28
RM_COXA_ROLL = 29
RM_COXA_YAW = 30
RM_FEMUR = 31
RM_FEMUR_ROLL = 32
RM_TIBIA = 33
RM_TARSUS = 34

RH_COXA = 35
RH_COXA_ROLL = 36
RH_COXA_YAW = 37
RH_FEMUR = 38
RH_FEMUR_ROLL = 39
RH_TIBIA = 40
RH_TARSUS = 41

#### Set up simulation parameters

In [50]:
run_time = 1

sim_params = Parameters(timestep=1e-4,
    render_mode="saved",
    render_playspeed=0.2,
    enable_adhesion=True,
    draw_adhesion=True,
    render_camera="Animat/camera_left",
)

actuated_joints = flygym.mujoco.preprogrammed.all_leg_dofs

target_num_steps = int(run_time / sim_params.timestep) # 10'000


In [51]:
nmf = NeuromechflyProject(sim_params=sim_params,)

preprogrammed_steps = PreprogrammedSteps()

swing_periods = preprogrammed_steps.swing_period

legs = preprogrammed_steps.legs

actuated_joints = preprogrammed_steps.dofs_per_leg

standing_action = []

for leg in legs:
    if leg.endswith("M"):
        standing_action.extend(preprogrammed_steps.get_joint_angles(leg, swing_periods[leg][1]))
    else:
        standing_action.extend(preprogrammed_steps.get_joint_angles(leg, 0.0))

standing_action.extend([0]*5) # Add the abdomen actuators at zero

standing_action = {'joints': standing_action, "adhesion": np.zeros(len(legs))}

for i in range(int(0.2//nmf.timestep)):
    nmf.step(standing_action)

#### 1ST MOVEMENT: Move middle legs to the back (1st movement)

In [52]:
target_num_steps = 5000
nmf.reset()
foreleg_ids = np.zeros(target_num_steps)
middle_stance_ids = np.linspace(swing_periods["RM"][1], -1/4*np.pi, target_num_steps)
hind_swings_ids = np.zeros(target_num_steps)

adhesion_action = np.array([1.0 if leg.endswith("F") else 0.0 for leg in legs])
all_joint_angles = []

for i in trange(target_num_steps):
    joint_angles = []
    for leg in legs:
        if leg.endswith("H"):
            joint_angles.extend(preprogrammed_steps.get_joint_angles(leg, hind_swings_ids[i]))
        elif leg.endswith("M"):
            joint_angles.extend(preprogrammed_steps.get_joint_angles(leg, middle_stance_ids[i]))
        else:
            joint_angles.extend(preprogrammed_steps.get_joint_angles(leg, foreleg_ids[i]))
    joint_angles.extend([0]*5) # Add abdomen actuators at zero

    all_joint_angles.append(joint_angles.copy())
    
    action = {'joints': np.array(joint_angles), "adhesion": adhesion_action}
    nmf.step(action)
    nmf.render()
nmf.save_video("./outputs/b.mp4")

# Save the last joint angles
last_joint_angles = all_joint_angles[-1]
all_joint_angles_arr = np.array(all_joint_angles)

100%|██████████| 5000/5000 [00:19<00:00, 253.22it/s]


#### 2ND MOVEMENT: Weight on front and middle legs

In [53]:
sim_params = Parameters(timestep=1e-4,
    render_mode="saved",
    render_playspeed=0.2,
    enable_adhesion=True,
    draw_adhesion=True,
    render_camera="Animat/camera_top",
)

target_num_steps = 5000
half_nbre_steps = target_num_steps//2
nmf = NeuromechflyProject(sim_params=sim_params,)

In [54]:
abdomen_ctrl_up = np.zeros((target_num_steps)) # shape (nbre_steps, )
abdomen_ctrl_up[:half_nbre_steps] = np.linspace(0, np.pi/20, half_nbre_steps)
abdomen_ctrl_up[half_nbre_steps:] = np.ones(target_num_steps-half_nbre_steps)*np.pi/20 # abd is up after 1 seconds and stays up
abd_joint = np.tile(abdomen_ctrl_up, (5, 1)).T

In [55]:
nmf.reset()

all_joint_angles_2nd = np.zeros([target_num_steps,47]) # shape (nbre_steps, 47)
print(all_joint_angles_2nd.shape)

print(len(last_joint_angles))

for i in range(len(last_joint_angles)):
#### LEFT LEGS ####    
    if i == LF_TIBIA: # LF tibia
        all_joint_angles_2nd[:,i] = np.linspace(last_joint_angles[LF_TIBIA], 1/3*np.pi, target_num_steps)
    elif i == LF_TARSUS : # LF tarsus
        all_joint_angles_2nd[:,i] = np.linspace(last_joint_angles[LF_TARSUS], 0, target_num_steps)

    elif i == LM_TIBIA: # LM tibia
        all_joint_angles_2nd[:,i] = np.linspace(last_joint_angles[LM_TIBIA], 1/3*np.pi, target_num_steps)
    elif i == LM_TARSUS: # LM tarsus
        all_joint_angles_2nd[:,i] = np.linspace(last_joint_angles[LM_TARSUS], 0, target_num_steps)
    elif i == LM_FEMUR: # LM FEMUR
        all_joint_angles_2nd[:,i] = np.linspace(last_joint_angles[LM_FEMUR], -3/7*np.pi, target_num_steps)
    elif i == LM_FEMUR_ROLL: # LM FEMUR ROLL
        all_joint_angles_2nd[:,i] = np.linspace(last_joint_angles[LM_FEMUR_ROLL], last_joint_angles[LM_FEMUR_ROLL]/2, target_num_steps)
    # elif i == LM_COXA: # LM COXA
    #     all_joint_angles_2nd[:,i] = np.linspace(last_joint_angles[LM_COXA], 0, target_num_steps)

    elif i == LH_FEMUR: # LH FEMUR
        all_joint_angles_2nd[:,i] = np.linspace(last_joint_angles[LH_FEMUR], -2/5*np.pi, target_num_steps)
    elif i == LH_TIBIA: # LH tibia
        all_joint_angles_2nd[:,i] = np.linspace(last_joint_angles[LH_TIBIA], 3*np.pi/5, target_num_steps)
    elif i == LH_TARSUS: # LH tarsus
        all_joint_angles_2nd[:,i] = np.linspace(last_joint_angles[LH_TARSUS], -np.pi/12, target_num_steps)

#### RIGHT LEGS ####
    elif i == RF_TIBIA: # RF tibia
        all_joint_angles_2nd[:,i] = np.linspace(last_joint_angles[RF_TIBIA], 1/3*np.pi, target_num_steps)
    elif i == RF_TARSUS: # RF tarsus
        all_joint_angles_2nd[:,i] = np.linspace(last_joint_angles[RF_TARSUS], 0, target_num_steps)

    elif i == RM_TIBIA: # RM tibia
        all_joint_angles_2nd[:,i] = np.linspace(last_joint_angles[RM_TIBIA], 1/3*np.pi, target_num_steps)
    elif i == RM_TARSUS: # RM tarsus
        all_joint_angles_2nd[:,i] = np.linspace(last_joint_angles[RM_TARSUS], 0, target_num_steps)
    elif i == RM_FEMUR: # LM FEMUR
        all_joint_angles_2nd[:,i] = np.linspace(last_joint_angles[RM_FEMUR], -3/7*np.pi, target_num_steps)
    elif i == RM_FEMUR_ROLL: # RM FEMUR ROLL
        all_joint_angles_2nd[:,i] = np.linspace(last_joint_angles[RM_FEMUR_ROLL], last_joint_angles[RM_FEMUR_ROLL]/2, target_num_steps)
    # elif i == RM_COXA: # RM COXA
    #     all_joint_angles_2nd[:,i] = np.linspace(last_joint_angles[RM_COXA], 0, target_num_steps)
    
    elif i == RH_FEMUR: # RH FEMUR
        all_joint_angles_2nd[:,i] = np.linspace(last_joint_angles[RH_FEMUR], -2/5*np.pi, target_num_steps)
    elif i == RH_TIBIA: # RH tibia
        all_joint_angles_2nd[:,i] = np.linspace(last_joint_angles[RH_TIBIA], 3*np.pi/5, target_num_steps)
    elif i == RH_TARSUS: # RH tarsus
        all_joint_angles_2nd[:,i] = np.linspace(last_joint_angles[RH_TARSUS], -np.pi/12, target_num_steps)
                    
    else:
        all_joint_angles_2nd[:,i] = np.linspace(last_joint_angles[i], last_joint_angles[i], target_num_steps)

all_joint_angles_2nd[:,42:] = abd_joint
adhesion_action = np.array([0.0 if leg.endswith("H") else 1.0 for leg in legs])

for i in trange(target_num_steps):
    joint = all_joint_angles_2nd[i,:]
    action = {'joints': joint, "adhesion": adhesion_action}
    nmf.step(action)
    nmf.render()
last_joint_angles_2nd = all_joint_angles_2nd[-1,:]
nmf.save_video("./outputs/c.mp4")

(5000, 47)
47


100%|██████████| 5000/5000 [00:21<00:00, 228.20it/s]


#### Actuate the hindlegs

In [56]:
nbre_steps = 5000
all_joint_angles_abd = np.tile(last_joint_angles_2nd,(nbre_steps,1)) # shape (nbre_steps, 47)

# Update all_joint_angles_abd with the new joint angles of leg 
start_LH_COXA_YAW = last_joint_angles_2nd[LH_COXA_YAW] 
start_RH_COXA_YAW = last_joint_angles_2nd[RH_COXA_YAW] 
start_LH_COXA = last_joint_angles_2nd[LH_COXA] 
start_RH_COXA = last_joint_angles_2nd[RH_COXA] 
start_LH_FEMUR_ROLL = last_joint_angles_2nd[LH_FEMUR_ROLL]
start_RH_FEMUR_ROLL = last_joint_angles_2nd[RH_FEMUR_ROLL]
start_LH_FEMUR = last_joint_angles_2nd[LH_FEMUR] 
start_RH_FEMUR = last_joint_angles_2nd[RH_FEMUR] 
start_LH_TARSUS = last_joint_angles_2nd[LH_TARSUS] 
start_RH_TARSUS = last_joint_angles_2nd[RH_TARSUS] 

# Spread hindlegs
all_joint_angles_abd[:,LH_COXA_YAW] = np.linspace(start_LH_COXA_YAW, start_LH_COXA_YAW-2*np.pi/5, nbre_steps)
all_joint_angles_abd[:,RH_COXA_YAW] = np.linspace(start_RH_COXA_YAW, start_RH_COXA_YAW+2*np.pi/5, nbre_steps)

all_joint_angles_abd[:,LH_FEMUR_ROLL] = np.linspace(start_LH_FEMUR_ROLL, start_LH_FEMUR_ROLL-np.pi/4, nbre_steps)
all_joint_angles_abd[:,RH_FEMUR_ROLL] = np.linspace(start_RH_FEMUR_ROLL, start_RH_FEMUR_ROLL+np.pi/4, nbre_steps)

# Up hindlegs
# all_joint_angles_abd[:,LH_COXA] = np.linspace(start_LH_COXA, start_LH_COXA+np.pi/6, nbre_steps)
# all_joint_angles_abd[:,RH_COXA] = np.linspace(start_RH_COXA, start_RH_COXA+np.pi/6, nbre_steps)

all_joint_angles_abd[:,LH_FEMUR] = np.linspace(start_LH_FEMUR, start_LH_FEMUR - np.pi/6, nbre_steps)
all_joint_angles_abd[:,RH_FEMUR] = np.linspace(start_RH_FEMUR, start_RH_FEMUR - np.pi/6, nbre_steps)

# Spread tarsus
all_joint_angles_abd[:,LH_TARSUS] = np.linspace(start_LH_TARSUS, start_LH_TARSUS-np.pi/3, nbre_steps)
all_joint_angles_abd[:,RH_TARSUS] = np.linspace(start_RH_TARSUS, start_RH_TARSUS-np.pi/3, nbre_steps)


##### Simulation

In [57]:
adhesion_action = np.array([0.0 if leg.endswith("H") else 1.0 for leg in legs])

for i in trange(nbre_steps):
    joint_angles = all_joint_angles_abd[i,:]
    action = {'joints': joint_angles, "adhesion": adhesion_action}
    nmf.step(action)
    nmf.render()

if sim_params.render_camera == "Animat/camera_back":
    nmf.save_video("./outputs/up_abd_back.mp4")
elif sim_params.render_camera == "Animat/camera_left":
    nmf.save_video("./outputs/up_abd_left.mp4")
elif sim_params.render_camera == "Animat/camera_right":
    nmf.save_video("./outputs/up_abd_right.mp4")
elif sim_params.render_camera == "Animat/camera_front":    
    nmf.save_video("./outputs/up_abd_front.mp4")
elif sim_params.render_camera == "Animat/camera_top":
    nmf.save_video("./outputs/up_abd_top.mp4")
elif sim_params.render_camera == "Animat/camera_bottom":
    nmf.save_video("./outputs/up_abd_bottom.mp4")

last_joint_angles_abd_up = all_joint_angles_abd[-1].copy()

100%|██████████| 5000/5000 [00:37<00:00, 131.79it/s]


#### Groom abdomen
1. move up un peu l'abdomen
2. check les dernières positions des hindlegs
3. écarter les pattes arrière ---> changer COXA YAW
4. lever les pattes arrière ---> changer FEMUR PITCH
5. abaisser (à plat) la pattes arrière ---> changer FEMUR ROLL
6. 

#### Move the hind legs up

#### Load grooming position of frontleg

In [58]:
# grooming_module_path = Path("./data/grooming_modules_provided_slow.pkl")
# with open(grooming_module_path, "rb") as f:
#     grooming_modules = pickle.load(f)

# timestep_groom = grooming_modules["timestep"]
# target_num_steps_groom = int(1.0 / timestep_groom) # 10'000
# target_joint_angles_front = grooming_modules["foreleg"][:, :target_num_steps]


In [59]:
# # all_joint_angles.shape() = (10000, 42)
# # my all_joint_angles_groom_shape() = (558, 21)

# # need to map the 21 joints to the 42 joints
# target_joint_angles_front = target_joint_angles_front.T # (558, 21)
# target_joint_angles_front = np.tile(target_joint_angles_front, (18, 1)) # (10044, 21)

# joint_LHCoxa = target_joint_angles_front[:,3]

In [60]:
# print(joint_LHCoxa.shape)
# print(joint_LHCoxa[:target_num_steps].shape)
# test = np.ones((target_num_steps, len(all_joint_angles[0])))
# print(test.shape)
# print(target_num_steps)

#### Map the grooming position of the leg to the corresponding body part.

In [61]:
# joint_LHCoxa = target_joint_angles_front[:,3]
# joint_LHCoxa_roll = target_joint_angles_front[:,4]
# joint_LHCoxa_yaw = target_joint_angles_front[:,5]
# joint_LHFemur = target_joint_angles_front[:,6]
# joint_LHFemur_roll = target_joint_angles_front[:,7]
# joint_LHTibia = target_joint_angles_front[:,8]
# joint_LHTarsus = target_joint_angles_front[:,9]
# joint_RHCoxa = target_joint_angles_front[:,10]
# joint_RHCoxa_roll = target_joint_angles_front[:,11]
# joint_RHCoxa_yaw = target_joint_angles_front[:,12]
# joint_RHFemur = target_joint_angles_front[:,13]
# joint_RHFemur_roll = target_joint_angles_front[:,14]
# joint_RHTibia = target_joint_angles_front[:,15]
# joint_RHTarsus = target_joint_angles_front[:,16]


# all_joint_angles = np.ones((target_num_steps, len(all_joint_angles[0]))) # 10000, 42
# print(all_joint_angles.shape)

# all_joint_angles[:,14] = joint_LHCoxa[:target_num_steps] # jusqu'à 10'000
# all_joint_angles[:,15] = joint_LHCoxa_roll[:target_num_steps]
# all_joint_angles[:,16] = joint_LHCoxa_yaw[:target_num_steps]
# all_joint_angles[:,17] = joint_LHFemur[:target_num_steps]
# all_joint_angles[:,18] = joint_LHFemur_roll[:target_num_steps]
# all_joint_angles[:,19] = joint_LHTibia[:target_num_steps]
# all_joint_angles[:,20] = joint_LHTarsus[:target_num_steps]
# all_joint_angles[:,35] = joint_RHCoxa[:target_num_steps]
# all_joint_angles[:,36] = joint_RHCoxa_roll[:target_num_steps]
# all_joint_angles[:,37] = joint_RHCoxa_yaw[:target_num_steps]
# all_joint_angles[:,38] = joint_RHFemur[:target_num_steps]
# all_joint_angles[:,39] = joint_RHFemur_roll[:target_num_steps]
# all_joint_angles[:,40] = joint_RHTibia[:target_num_steps]
# all_joint_angles[:,41] = joint_RHTarsus[:target_num_steps]
